In [ ]:
import os
import numpy as np
import pandas as pd
import tqdm
import cv2
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

In [ ]:
PAD = True
WIDTH = 1024
HEIGHT = 1024

In [ ]:
data_folder = "/kaggle/input/hotel-id-2021-fgvc8/"
train_df = pd.read_csv(data_folder + "train.csv")

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img


def open_and_preprocess_image(row_df):
    img = cv2.imread(f"{data_folder}train_images/{row_df.chain.astype(int)}/{row_df.image}")
    if PAD:
        img = pad_image(img)    
    
    return cv2.resize(img, (WIDTH, HEIGHT))


def save_image(row_df, img):
    cv2.imwrite(f"{row_df.image}", img)
    
    
def process_row(row_df):
    img = open_and_preprocess_image(row_df)
    save_image(row_df, img)

In [ ]:
%%time
dfs_proc = Parallel(n_jobs=4, prefer='threads')(delayed(process_row)(train_df.loc[i]) for i in range(0, len(train_df)))

In [ ]:
# !cd /kaggle/working/ & zip -jqr train.zip .
# !find . -name "*.jpg" -delete